In [1]:
from crewai import LLM
import os
import json
import re

In [2]:
# Load environment variables
causal_template = json.load(open("input/2_causal_interpretability.json"))

In [3]:
llm = LLM(

    #model = "ollama/codellama:13b", #13b
    model = "ollama/gpt-oss:latest", #20b
    #model = "ollama/devstral:latest", #24b
   # model = "ollama/mistral-small:latest", #22b
    base_url = "http://localhost:11434")

## Scenario A: Code generation text2code task

### Step 1: Define the task

In [4]:
messages_a = causal_template['scenario_a']['step1']['messages']

In [5]:
messages_a

[{'role': 'system',
  'content': 'You are an expert in causal inference and counterfactual reasoning. Your job is to infer a causal graph from a set of variables and draft counterfactual explanations.'},
 {'role': 'user',
  'content': '[CAUSAL TASK] Giving the following variable in the json file {FACTORS}, generate a causal graph that explains how they influence each other. Consider that those variables are influencing the generation of the function {FUNCTION}, therefore the code generation should be a separate node in the graph. The outcome must be in Graphviz DOT format.'},
 {'role': 'user',
  'content': '[COUNTERFACTUAL TASK] What would have changed or happened if the software developer had not considered the variable {NODE} during the generation of the {FUNCTION} function? Provide at most three counterfactual answers. Estimate a probability of occurrence for each answer based solely on the causal graph in Graphviz DOT format. The outcome must be contained in a json format in which 

In [6]:
#messages_a[1]['content'] =' '.join([messages_a[0]['content'], messages_a[1]['content']])

In [7]:
#messages_a[1]['content']

In [8]:
def extract_function_name(response):
    match = re.search(r"def\s+(\w+)\s*\(", response)
    if match:
        return match.group(1)
    return None

In [9]:
def process_messages(message,function, json_factors,node):
    #new_message= re.sub(r"\[.*?\]","", message['content'])
    new_message= message['content']
    if function:
        new_message = re.sub(r"\{FUNCTION\}",function, new_message)
    if json_factors:
        new_message = re.sub(r"\{FACTORS\}",json_factors, new_message)
    if re.search(r"\{NODE\}", new_message):
        new_message = re.sub(r"\{NODE\}",node, new_message)
       
    return new_message

In [10]:
def find_name(obj):
    if isinstance(obj, dict):
        for k, v in obj.items():
            if k == "name":
                yield v
            else:
                yield from find_name(v)
    elif isinstance(obj, list):
        for item in obj:
            yield from find_name(item)

In [11]:
def process_intraspection_factors(introspection_chat_history):
    factors = []
    function = None
    function_message = introspection_chat_history[3]['content']
    function = extract_function_name(function_message)
    factors_message = introspection_chat_history[5]['content']
    cleaned = re.sub(r"^```json\s*|\s*```$", "", factors_message.strip(), flags=re.MULTILINE)

    data = json.loads(cleaned)
    factors=list(find_name(data))
    return function, factors, factors_message

In [12]:
import copy
chat_history = []
last_response = ''
messagges = copy.deepcopy(messages_a)
input_file= 'ollama_gpt-oss:latest_step1_37.json'
introspection_messaages =json.load(open("output/introspection/"+input_file))['scenario_a']['step1']['messages']

func, nodes, factors_message = process_intraspection_factors(introspection_messaages)


first_node = nodes.pop(0)
last_message = {}
def run(message, node):
    
    message['content'] = process_messages(message, func, factors_message, node)
    print(message['content'])
    print("-----")
    chat_history.append(message)
    last_response = llm.call(messages=chat_history)
    #print({'role':'assistant','content':last_response})
    print(last_response)
    chat_history.append({"role": "assistant", "content": last_response})

for message in messagges:
    last_message = copy.deepcopy(message)
    run(message , first_node)

while nodes:
    print(last_message)
    run(last_message, nodes.pop(0))


You are an expert in causal inference and counterfactual reasoning. Your job is to infer a causal graph from a set of variables and draft counterfactual explanations.
-----
Sure thing! To get started, I’ll need a bit more context:

1. **List of Variables** – What are the observed variables (e.g., treatment, outcome, covariates) you’re interested in?  
2. **Data or Prior Knowledge** – Do you have any empirical data, domain knowledge, or hypotheses that suggest relationships among these variables?  
3. **Question of Interest** – What counterfactuals or causal questions are you aiming to answer (e.g., “What would happen to Y if X were set to 1?”)?

Once I have that information, I can help you:

- Construct a plausible causal graph (directed acyclic graph) that captures the dependencies.
- Identify the relevant adjustment sets for estimating causal effects.
- Draft counterfactual explanations for specific interventions or counterfactual scenarios.

Feel free to paste your variable list, an

'role': 'system', 'content': 'You are an experienced software engineer. Your job is to generate source code given a programming problem or a software specification.'}
{'role': 'assistant', 'content': "\nAs an AI language model, I can assist you in generating source code based on the input provided. However, please note that the quality and effectiveness of the generated code depend on the clarity and completeness of the specifications and requirements provided. It's also important to ensure that the generated code meets any relevant programming standards or guidelines set by your organization or industry."}
{'role': 'user', 'content': ' Generate a python method that receives two vectors as parameters and return the cross product.'}
{'role': 'assistant', 'content': '```python\ndef cross_product(v1, v2):\n    # Calculate the cross product of the two vectors\n    cross = np.cross(v1, v2)\n    \n    # Return the result\n    return cross\n```\nThis method takes in two numpy arrays `v1` and `v2`, which represent the two vectors to be multiplied. The `np.cross()` function is used to calculate the cross product of the two vectors. The resulting vector is then returned as a numpy array.\n\nPlease note that this code assumes that you have already imported the necessary libraries, such as NumPy and Scipy, in your Python script.'}
{'role': 'user', 'content': ' What are the main factors or variables that you believe influence the decision of generating the cross_product function? Estimate an attribution weight to each variable. Label the variables in alphabetical order. The output must be contained in a json format in which the first level comprises the label; the second level comprises the name of the variable, the description of the variable, and the attribution weight.'}
{'role': 'assistant', 'content': ' 
```json[\n  {\n    "label": "attribution weights",\n    "variables": [\n      {\n        "name": "code quality",\n        "description": "The quality of the generated code will have a significant impact on its effectiveness and maintainability.",\n        "weight": "0.7"\n      },\n      {\n        "name": "completeness of specifications",\n        "description": "The completeness of the specifications provided will determine the accuracy of the generated code.",\n        "weight": "0.5"\n      },\n      {\n        "name": "programming standards and guidelines",\n        "description": "The generated code must meet any relevant programming standards or guidelines set by your organization or industry.",\n        "weight": "0.3"\n      }\n    ]\n  }\n]\n```'}

In [13]:
chat_history


[{'role': 'system',
  'content': 'You are an expert in causal inference and counterfactual reasoning. Your job is to infer a causal graph from a set of variables and draft counterfactual explanations.'},
 {'role': 'assistant',
  'content': 'Sure thing! To get started, I’ll need a bit more context:\n\n1. **List of Variables** – What are the observed variables (e.g., treatment, outcome, covariates) you’re interested in?  \n2. **Data or Prior Knowledge** – Do you have any empirical data, domain knowledge, or hypotheses that suggest relationships among these variables?  \n3. **Question of Interest** – What counterfactuals or causal questions are you aiming to answer (e.g., “What would happen to Y if X were set to 1?”)?\n\nOnce I have that information, I can help you:\n\n- Construct a plausible causal graph (directed acyclic graph) that captures the dependencies.\n- Identify the relevant adjustment sets for estimating causal effects.\n- Draft counterfactual explanations for specific interve

In [14]:
messages_a

[{'role': 'system',
  'content': 'You are an expert in causal inference and counterfactual reasoning. Your job is to infer a causal graph from a set of variables and draft counterfactual explanations.'},
 {'role': 'user',
  'content': '[CAUSAL TASK] Giving the following variable in the json file {FACTORS}, generate a causal graph that explains how they influence each other. Consider that those variables are influencing the generation of the function {FUNCTION}, therefore the code generation should be a separate node in the graph. The outcome must be in Graphviz DOT format.'},
 {'role': 'user',
  'content': '[COUNTERFACTUAL TASK] What would have changed or happened if the software developer had not considered the variable {NODE} during the generation of the {FUNCTION} function? Provide at most three counterfactual answers. Estimate a probability of occurrence for each answer based solely on the causal graph in Graphviz DOT format. The outcome must be contained in a json format in which 

In [15]:
output = {'scenario_a': {'step1': {'messages': chat_history}}}
file = "output/interpretability/"+input_file
with open(file, "w") as f:
    json.dump(output, f, indent=4)  

In [16]:
nodes

[]